In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/NLP/project


In [3]:
!pip install -r requirements.txt

# Train with No Embeddings

In [4]:
!python run_lstm.py \
  --username "elonmusk" \
  --num_epochs 7 \
  --eval_every 2000 \
  --hidden_size 512 \
  --embedding_dim 300 \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=7, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embeddings=False, username='elonmusk')
12/03/2021 20:11:11 - INFO - __main__ -   [Iter 2000] Loss 7.05  Val Loss: 6.84  Val Perplexity: 935.69
12/03/2021 20:11:19 - INF

In [5]:
!python run_lstm.py \
  --username "karpathy" \
  --num_epochs 7 \
  --eval_every 2000 \
  --hidden_size 512 \
  --embedding_dim 300 \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=7, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embeddings=False, username='karpathy')
12/03/2021 20:12:33 - INFO - __main__ -   [Iter 2000] Loss 7.13  Val Loss: 7.18  Val Perplexity: 1307.88
12/03/2021 20:12:43 - IN

In [6]:
!python run_lstm.py \
  --username "BarackObama" \
  --num_epochs 7 \
  --eval_every 2000 \
  --hidden_size 512 \
  --embedding_dim 300 \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 6000     /        0
[00:00:00] Pre-processing sequences                 ████████ 13000    /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=7, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embedd

In [7]:
!python run_lstm.py \
  --username "cnn" \
  --num_epochs 7 \
  --eval_every 2000 \
  --hidden_size 512 \
  --embedding_dim 300 \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 12000    /        0
[00:00:00] Pre-processing sequences                 ████████ 22000    /        0
[00:00:00] Pre-processing sequences                 ████████ 29000    /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, nu

In [8]:
!python run_lstm.py \
  --username "wsj" \
  --num_epochs 7 \
  --eval_every 2000 \
  --hidden_size 512 \
  --embedding_dim 300 \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 9000     /        0
[00:00:00] Pre-processing sequences                 ████████ 18000    /        0
[00:00:00] Pre-processing sequences                 ████████ 26000    /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, nu

## Generation

In [14]:
!python run_lstm_generation.py \
  --username "elonmusk" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 21:27:05 - INFO - __main__ -   Sample 0: Tesla software V10. 0 comes with karaoke
12/03/2021 21:27:05 - INFO - __main__ -   Sample 1: 
12/03/2021 21:27:05 - INFO - __main__ -   Sample 2: Tesla Supercharger network now energized from Cape Canaveral to California, but the second derivative bodes on the droneship. Just need the future.
12/03/2021 21:27:05 - INFO - __main__ -   Sample 3: Just leaving SpaceX Starship build site in Boca
12/03/2021 21:27:05 - INFO - __main__ -   Sample 4: Will be live streaming the Gigafactory launch party tonight. Check this eve.
12/03/2021 21:27:05 - INFO - __main__ -   Sample 5: Falcon Heavy goes vertical
12/03/2021 21:27:05 - INFO - __main__ -   Sample 6: Falcon Heavy goes vertical
12/03/2021 21:27:05 - INFO - __main__ -   Sample 7: At Tesla Giga Shanghai NSFW!!
12/03/2021 21:27:05 - INFO - __main__ -   Sample 8: Tesla deliveries & AP hardware chart by @ lexfridman at MIT
12/03/2021 21:27:05 - INFO - __main__ -   Sample 9: Just leaving SpaceX S

In [15]:
!python run_lstm_generation.py \
  --username "karpathy" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 21:27:10 - INFO - __main__ -   Sample 0: Awesome : " The Drill of Depth " ( Portable Kinect 3D Camera ) from David Cox lab ( @ neurobongo )
12/03/2021 21:27:10 - INFO - __main__ -   Sample 1: At last, Stanford presentation. Fun, will look at all. I like the same direction. " 1
12/03/2021 21:27:10 - INFO - __main__ -   Sample 2: This is a motherfucking website. the guy has a point
12/03/2021 21:27:10 - INFO - __main__ -   Sample 3: I hate a new new trend about designs with ConvNetJS from a nice MLdata science blog : & video of your pet with Robots
12/03/2021 21:27:10 - INFO - __main__ -   Sample 4: One of my favorite funny YouTube clips : " Everythings Amazing & Nobodys Happy " ( fun starts ~) 1
12/03/2021 21:27:10 - INFO - __main__ -   Sample 5: Wow pretending to be blind is Really hard! 1
12/03/2021 21:27:10 - INFO - __main__ -   Sample 6: I know that take 1
12/03/2021 21:27:10 - INFO - __main__ -   Sample 7: # icra2013 robots batch two vine. co Andrej Karpathy' s post on V

In [16]:
!python run_lstm_generation.py \
  --username "BarackObama" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 21:27:15 - INFO - __main__ -   Sample 0: President Obama : “ I ’ ll use the money we ’ re fighting to create the future that ’ s not going.”
12/03/2021 21:27:15 - INFO - __main__ -   Sample 1: President Obama : " We have to renew our military around the world in our commitment to fight back against all of us.”
12/03/2021 21:27:15 - INFO - __main__ -   Sample 2: “ We believe we have a strong, middle - class opportunity for all that hard work and responsibility is rewarded.”— President Obama
12/03/2021 21:27:15 - INFO - __main__ -   Sample 3: FACT : Obama' s plan to reduce the deficit by $ 4 trillion to $ 4 trillion in 10 years. # BuffettRule
12/03/2021 21:27:15 - INFO - __main__ -   Sample 4: Have you heard?
12/03/2021 21:27:15 - INFO - __main__ -   Sample 5: Read the latest op - ed from @ nytimes : " We can bring the job with a balanced plan that would make it harder for our people and our future."
12/03/2021 21:27:15 - INFO - __main__ -   Sample 6: President Obama : “ I am 

In [17]:
!python run_lstm_generation.py \
  --username "cnn" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 21:27:20 - INFO - __main__ -   Sample 0: Opinion : An e - commerce is likely to be a budget, writes @ jasonrileywsj
12/03/2021 21:27:20 - INFO - __main__ -   Sample 1: After years of a man, the attack, was the deaths of violence in the past four days of a attack on June 2012.
12/03/2021 21:27:20 - INFO - __main__ -   Sample 2: How to put the rule of violence
12/03/2021 21:27:20 - INFO - __main__ -   Sample 3: The U. S. has backed since the U. S. to five years ago. Will the new set of new rules?
12/03/2021 21:27:20 - INFO - __main__ -   Sample 4: :' The talk of' has been for them.
12/03/2021 21:27:20 - INFO - __main__ -   Sample 5: GM to Settle SEC America
12/03/2021 21:27:20 - INFO - __main__ -   Sample 6: As the world, we' re make a comeback :
12/03/2021 21:27:20 - INFO - __main__ -   Sample 7: For your neighborhood, being a slowing like a lot by " no " too much.
12/03/2021 21:27:20 - INFO - __main__ -   Sample 8: Opinion : How to stop the ones of the NFL in the House tax b

In [18]:
!python run_lstm_generation.py \
  --username "wsj" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 21:27:25 - INFO - __main__ -   Sample 0: Man kills wife, self in Mount death 2
12/03/2021 21:27:25 - INFO - __main__ -   Sample 1: How did a # PoliticalTicker in a Twitter account of the @, and the the best and the sale of a # Rio2016 GIF
12/03/2021 21:27:25 - INFO - __main__ -   Sample 2: Man admits killing son, mother' s' found dead after 3
12/03/2021 21:27:25 - INFO - __main__ -   Sample 3: A school teacher, the man who helped her home with two weeks after decades accident.
12/03/2021 21:27:25 - INFO - __main__ -   Sample 4: " A - Rod is a missing of a couple that will be on a large via @ SarahHoyeCNN
12/03/2021 21:27:25 - INFO - __main__ -   Sample 5: New York police have some ice resume for fall for # CNNdebate?
12/03/2021 21:27:25 - INFO - __main__ -   Sample 6: The top 10 tech ‘ fails ’ of 2011.
12/03/2021 21:27:25 - INFO - __main__ -   Sample 7: What' s on the Apple' s museum could be the next few hours before home before they may have been kidnapped on # CNNElection

# Train With GloVe Embeddings

In [50]:
!python run_lstm.py \
  --username "elonmusk" \
  --num_epochs 5 \
  --eval_every 2000 \
  --hidden_size 512 \
  --use_pretrained_embeddings True \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=5, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embeddings=True, username='elonmusk')
12/03/2021 21:41:59 - INFO - __main__ -   Using pretrained twitter GloVe embeddings...
12/03/2021 21:41:59 - INFO - torchtext.voca

In [51]:
!python run_lstm.py \
  --username "karpathy" \
  --num_epochs 5 \
  --eval_every 2000 \
  --hidden_size 512 \
  --use_pretrained_embeddings True \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=5, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embeddings=True, username='karpathy')
12/03/2021 21:43:02 - INFO - __main__ -   Using pretrained twitter GloVe embeddings...
12/03/2021 21:43:02 - INFO - torchtext.voca

In [52]:
!python run_lstm.py \
  --username "BarackObama" \
  --num_epochs 5 \
  --eval_every 2000 \
  --hidden_size 512 \
  --use_pretrained_embeddings True \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 7000     /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=2000, hidden_size=512, length=50, lr=0.0004, n_gpu=1, n_layers=1, no_cuda=False, num_epochs=5, output_dir='outputs', seed=42, train_split=0.8, use_pretrained_embeddings=True, username='BarackObama')
12/03/2021 21:44:31 - INFO - __main__ -   Usin

In [53]:
!python run_lstm.py \
  --username "cnn" \
  --num_epochs 5 \
  --eval_every 2000 \
  --hidden_size 512 \
  --use_pretrained_embeddings True \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 9000     /        0
[00:00:00] Pre-processing sequences                 ████████ 19000    /        0
[00:00:00] Pre-processing sequences                 ████████ 27000    /        0
[00:00:00] Pre-processing sequences                 ████████ 31000    /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=20

In [54]:
!python run_lstm.py \
  --username "wsj" \
  --num_epochs 5 \
  --eval_every 2000 \
  --hidden_size 512 \
  --use_pretrained_embeddings True \
  --lr 0.0004

/gdrive/My Drive/NLP/project/dataset.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter]
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 9000     /        0
[00:00:00] Pre-processing sequences                 ████████ 17000    /        0
[00:00:00] Pre-processing sequences                 ████████ 23000    /        0
[00:00:00] Pre-processing sequences                 ████████ 28000    /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Namespace(bos_token='<BOS>', dropout=0.2, embedding_dim=300, eos_token='<EOS>', eval_every=20

## Generation

In [55]:
!python run_lstm_generation.py \
  --username "elonmusk" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 22:29:34 - INFO - __main__ -   Sample 0: At test flight legs about climate on Monday. This is real →?
12/03/2021 22:29:34 - INFO - __main__ -   Sample 1: 
12/03/2021 22:29:34 - INFO - __main__ -   Sample 2: Tesla Model X video a release Powerwall car ever in the Atlantic
12/03/2021 22:29:34 - INFO - __main__ -   Sample 3: 
12/03/2021 22:29:34 - INFO - __main__ -   Sample 4: Landing Tesla patents
12/03/2021 22:29:34 - INFO - __main__ -   Sample 5: First video rocket factory in Texas
12/03/2021 22:29:34 - INFO - __main__ -   Sample 6: The first launch is deleting live in Instructions )
12/03/2021 22:29:34 - INFO - __main__ -   Sample 7: My upgrade details
12/03/2021 22:29:34 - INFO - __main__ -   Sample 8: Model 3 Performance
12/03/2021 22:29:34 - INFO - __main__ -   Sample 9: I want I can never


In [56]:
!python run_lstm_generation.py \
  --username "karpathy" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 22:29:44 - INFO - __main__ -   Sample 0: A company imagines of my nostalgia and Deep Learning from Joan Bruna on the trenches # icra2013 # sorrow 1
12/03/2021 22:29:44 - INFO - __main__ -   Sample 1: This is what for a bit but I' m starting a very corner. I like a bug for the PhD liaison :
12/03/2021 22:29:44 - INFO - __main__ -   Sample 2: Nice roundup of my nostalgia from Google talk by MIT CSAIL I can a lot that of the research here of code favorites :
12/03/2021 22:29:44 - INFO - __main__ -   Sample 3: I put up a bit more articles from 1800msfwdbwd than a lot of ago, I' m trying to realize that with his. Fun to train
12/03/2021 22:29:44 - INFO - __main__ -   Sample 4: Working one of a cat of the brain in the politics of entertainment
12/03/2021 22:29:44 - INFO - __main__ -   Sample 5: Also a nice idea from ECCV on a day - scale manufacturing. I want to spread on Vine, but I' m a few bug of 1
12/03/2021 22:29:44 - INFO - __main__ -   Sample 6: I feel all the new ones. I h

In [57]:
!python run_lstm_generation.py \
  --username "BarackObama" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 22:29:53 - INFO - __main__ -   Sample 0: President Obama : " We know what change looks like because of the most important that has come roaring to be a." - passed President Obama
12/03/2021 22:29:53 - INFO - __main__ -   Sample 1: " We' ve got to make sure that every American is heard when they' ve got to work for us all gun." — President Obama
12/03/2021 22:29:53 - INFO - __main__ -   Sample 2: “ We know that we can be as one nation we can keep moving forward. We ’ re going to get out the vote this year." # SOTU
12/03/2021 22:29:53 - INFO - __main__ -   Sample 3: At an " America' s second event meeting for President Obama in Chicago :
12/03/2021 22:29:53 - INFO - __main__ -   Sample 4: If you' re proud of what health care reform, add your name :
12/03/2021 22:29:53 - INFO - __main__ -   Sample 5: In New York, IN at the State of the Union. We watch. Watch live at
12/03/2021 22:29:53 - INFO - __main__ -   Sample 6: " I want to make sure every person, the American people can m

In [58]:
!python run_lstm_generation.py \
  --username "cnn" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 22:30:02 - INFO - __main__ -   Sample 0: Obama to call for school assault.
12/03/2021 22:30:02 - INFO - __main__ -   Sample 1: Judge OKs Casey Anthony released from hospital after collapse - year - old.
12/03/2021 22:30:02 - INFO - __main__ -   Sample 2: What ’ s your favorite with? @ Bourdain goes to the new few. He' s # gottadoit # vs. @ AC360
12/03/2021 22:30:02 - INFO - __main__ -   Sample 3: Police : An of' d boy' to be shot, 1 dead in 2
12/03/2021 22:30:02 - INFO - __main__ -   Sample 4: ' Study new' for action.
12/03/2021 22:30:02 - INFO - __main__ -   Sample 5: Obama : Clinton in potentially with U. S. official says.
12/03/2021 22:30:02 - INFO - __main__ -   Sample 6: First, is speaking now at the Olympics. @ Bourdain is on # January tonight, 9pm ET.
12/03/2021 22:30:02 - INFO - __main__ -   Sample 7: ' Passengers of Man' in Polls - death, after' s killing of dead, dies at 7
12/03/2021 22:30:02 - INFO - __main__ -   Sample 8: ' Pirates Is' for Obama on his speech.
12

In [59]:
!python run_lstm_generation.py \
  --username "wsj" \
  --num_sequences 10 \
  --temperature 0.8 \
  --k 50 \
  --p 0.9 \
  --seed 52

12/03/2021 22:30:11 - INFO - __main__ -   Sample 0: Opinion : The SEC behind shooting back on survive ’ s capital - - Russia rule for an IPO
12/03/2021 22:30:11 - INFO - __main__ -   Sample 1: The new rate - service for an hour of California :
12/03/2021 22:30:11 - INFO - __main__ -   Sample 2: New York Times s new technology to pay up their 10 - year - old on the market.
12/03/2021 22:30:11 - INFO - __main__ -   Sample 3: The U. S. has is since a deal to lead the Fed ’ s largest companies - day after Sandy
12/03/2021 22:30:11 - INFO - __main__ -   Sample 4: Breaking : U. S. Postal Service said it will buy interest rates low - wage tax overhaul
12/03/2021 22:30:11 - INFO - __main__ -   Sample 5: The battle for America' s most - largest - ever by a group of week
12/03/2021 22:30:11 - INFO - __main__ -   Sample 6: Opinion : Trump ’ s tax cuts in House would Senate the list of Fed markets leader
12/03/2021 22:30:11 - INFO - __main__ -   Sample 7: A group of the world' s largest tech compa